## 1.语句生成器

In [105]:
diary_grammar = """
sentence => sub_phrase act_phrase*
sub_phrase => adj* sub
act_phrase => prep_phrase verb
act_phrase* => act_phrase punc1 | act_phrase punc2 act_phrase*
prep_phrase => adv prep place
sub =>   我 |  小明 | 老师 | 姐姐
adj =>  有意思的 | 欢快的 | 难过的 | 可爱的
adj* => null | adj adj*
adv => 昨天 | 今天 | 上周
prep => 在 | 去 | 到
place => 公园 | 操场 | 学校 | 体育馆
verb => 打羽毛球  | 跑步  |  打篮球 | 踢足球
punc1 => 。
punc2 => ，
"""

In [106]:
direction_grammar='''
sentence => direction_phrase* place_phrase
direction_phrase => direction verb distance
direction_phrase* => direction_phrase punc1 | direction_phrase punc1 direction_phrase*
place_phrase => place location punc2
direction => 直走 | 左转 | 右转
verb => 步行 | 跑步 | 坐公交 | 骑车
distance => 100米 | 1000米 | 10分钟 | 1小时
place => 博物馆 | 图书馆 | 电影院 | 餐厅
location => 在马路对面 | 在右边 | 在左边
punc1 => ，
punc2 => 。
'''

In [107]:
def create_grammar(grammar_str, split='=>',line_split='\n'):
    grammar={}
    for line in grammar_str.split(line_split):
        if not line.strip(): continue
        exp, stmt=line.split(split)
        grammar[exp.strip()]=[s.split() for s in stmt.split('|')]
    return grammar

In [108]:
import random
choice=random.choice
def generate(grammar,target):
    if target not in grammar: return target
    components=[generate(grammar,t) for t in choice(grammar[target])]
    return ''.join(e  for e in components if e != 'null')
    

In [109]:
for i in range(10):
    print(generate(create_grammar(diary_grammar),'sentence'))
for i in range(10):    
    print(generate(create_grammar(direction_grammar),'sentence'))

有意思的难过的老师今天在公园打篮球，上周到公园跑步，昨天去体育馆打篮球。
可爱的可爱的有意思的老师昨天在公园跑步，今天到学校打羽毛球。
姐姐上周到体育馆打羽毛球。
老师今天到操场打羽毛球。
欢快的可爱的可爱的可爱的有意思的姐姐今天在操场跑步。
可爱的姐姐上周去公园踢足球。
欢快的姐姐上周去操场跑步，今天去学校踢足球。
欢快的姐姐今天去学校打羽毛球，今天到学校踢足球。
可爱的难过的我上周在公园打羽毛球。
我上周到公园跑步，昨天到学校打篮球。
左转坐公交1000米，直走骑车1000米，左转骑车1000米，电影院在马路对面。
左转坐公交100米，博物馆在右边。
左转坐公交10分钟，直走跑步1小时，博物馆在右边。
右转坐公交1小时，左转坐公交1小时，左转跑步10分钟，博物馆在右边。
左转骑车1小时，右转骑车1000米，图书馆在左边。
直走骑车10分钟，左转步行10分钟，图书馆在右边。
右转坐公交1000米，图书馆在右边。
直走跑步10分钟，左转骑车1小时，直走步行1000米，左转骑车10分钟，餐厅在马路对面。
右转跑步1小时，直走步行1000米，左转步行1小时，左转跑步1小时，电影院在马路对面。
右转跑步100米，餐厅在马路对面。


## 2.使用新数据完成语言模型训练

In [160]:
filename='E:/AI/lesson1/movie_comments.csv'

In [165]:
import pandas as pd
import re
from collections import Counter
import jieba
content=pd.read_csv(filename)
articles=content['comment'].tolist()

C:\Users\zwconnie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [167]:
def token(string):
    return re.findall('\w+',string)

In [168]:
articles_clean=[''.join(token(str(a))) for a in articles]
TOKEN=[]
for a in articles_clean:
    TOKEN += list(jieba.cut(a))
TOKEN_2gram=[''.join(TOKEN[i:i+2]) for i in range(len(TOKEN)-2)]
words_count_2gram=Counter(TOKEN_2gram)
def prob_2gram(word1,word2):
    if word1+word2 in words_count_2gram:
        return words_count_2[word1+word2]/len(TOKEN_2gram)
    else:
        return 1/len(TOKEN_2gram)

In [169]:
def get_prob(sentence):
    words=list(jieba.cut(sentence))
    sentence_prob=1
    for i,word in enumerate(words[:-1]):
        next_=words[i+1]
        prob=prob_2gram(word,next_)
        sentence_prob *=prob
    return sentence_prob

## 3.获得最优质的语言

In [170]:
def generate_best(grammar, target, n):
    sentence_pool=[]
    sentence_cleanpool=[]
    for i in range(n):
        sentence=generate(create_grammar(grammar),target)
        sentence_clean=''.join(token(str(sentence)))
        sentence_pool.append((sentence,sentence_clean))
        sentence_cleanpool.append(sentence_clean)
    sentence_prob=[(get_prob(s),s) for s in sentence_cleanpool]
    sentence_prob=sorted(sentence_prob,key=lambda x:x[0],reverse=True)
    print('The most reasonable sentence is {} with probability {}'.format([sentence_pool[x][0] for x in range(len(sentence_pool)) if sentence_pool[x][1]==sentence_prob[0][1]][0],sentence_prob[0][0]))
    for prob,s in sentence_prob:
        print('-'*4 + ' {} with probability {}'.format([sentence_pool[x][0] for x in range(len(sentence_pool)) if sentence_pool[x][1]==s][0],prob))

In [171]:
generate_best(diary_grammar,'sentence',20)
generate_best(direction_grammar,'sentence',20)

The most reasonable sentence is 我今天到体育馆踢足球。 with probability 6.141497703553918e-26
---- 我今天到体育馆踢足球。 with probability 6.141497703553918e-26
---- 小明今天到公园踢足球。 with probability 6.823886337282131e-27
---- 难过的小明今天在学校跑步。 with probability 3.7260731991974255e-36
---- 有意思的我上周到操场跑步。 with probability 1.2735134650658724e-36
---- 可爱的小明上周到操场跑步。 with probability 1.5345936902984691e-38
---- 欢快的小明昨天在学校打羽毛球。 with probability 3.6969436530470894e-42
---- 我上周到公园踢足球，今天去公园打羽毛球。 with probability 5.06438285262883e-59
---- 欢快的我今天到学校打羽毛球，上周到学校打篮球。 with probability 8.740697846772398e-66
---- 姐姐今天到公园打羽毛球，昨天在学校跑步，今天去体育馆打羽毛球。 with probability 8.013245997775071e-88
---- 欢快的老师昨天在公园打篮球，昨天到学校打羽毛球，今天去学校打羽毛球。 with probability 0.0
---- 我今天在操场打篮球，今天在操场跑步，今天在体育馆打羽毛球，昨天到操场踢足球，今天到体育馆踢足球，昨天在操场跑步。 with probability 0.0
---- 老师今天去公园跑步，昨天到公园踢足球，上周到操场跑步。 with probability 0.0
---- 姐姐上周在学校踢足球，今天在体育馆跑步，上周去体育馆跑步，上周到公园打羽毛球，上周到操场打羽毛球，昨天去操场打羽毛球，上周在体育馆跑步，上周到公园打篮球。 with probability 0.0
---- 难过的欢快的有意思的欢快的姐姐今天在公园踢足球，今天去学校跑步，昨天到操场跑步。 with probab

#### Q: 这个模型有什么问题？ 你准备如何提升？

#### Ans: 
（1）生成语句的概率都很小，即使选出概率最大的句子也不能保证该句是所有语句中最合理的，需要进一步扩大语料库提升概率模型的准确度。
（2）目前的二元模型中的概率仅基于词组在语料库中的出现次数所计算，对于未出现词组，直接使用1/所有词组数来替代，需要进一步对零概率和小概率词组做出处理。
（3）以上句子的概率大小与句子长度直接相关，需要对句子长度标准化后再进行比较。